In [ ]:
%%capture

%pip install openai nltk ipywidgets numpy requests-cache backoff tiktoken nrclex pandas

# Sentiment Analysis with ChatGPT

While sentiment analysis is sort of like the ["Hello, world!"](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program#Variations) of [Natural Language Processing](https://en.wikipedia.org/wiki/Natural_language_processing) (NLP), luckily for us it's a bit more fun than just echoing out a string.

This notebook will introduce you to sentiment analysis using traditional NLP tools and then explore analyzing sentiment with [ChatGPT](https://openai.com/blog/chatgpt).

**Note**: For a better learning experience, this notebook contains some code cells that are only used to render widgets for you to interact with and some others that only generate data structures or variables that later cells will reference.

## What is sentiment analysis?

[Sentiment Analysis](https://en.wikipedia.org/wiki/Sentiment_analysis) is a way of analyzing some text to determine if it's positive, negative, or neutral.

This is the kind of thing that's pretty easy for a human who understands the language the text is written in to do, but it can be hard for a computer to really understand the underlying meaning behind the language.

### Examples

- "I saw that movie." (neutral)
- "I love that movie." (positive)
- "I hate that movie." (negative)


## Initial Setup

First, we'll import the relevant tools we'll be using in the notebook and configure some global variables.

- `nltk`: Python's [Natural Language Toolkit](https://www.nltk.org/), which we'll use to explore some more traditional sentiment analysis techniques
- `openai`: Python library for interacting with the [OpenAI API](https://platform.openai.com/docs/api-reference/introduction)

**Note**: In a later cell, we'll also make use of [`nrclex`](https://github.com/metalcorebear/NRCLex) to investigate some more advanced NLP, but because it's only used in one cell, we're importing it there for clarity.


In [ ]:
import os

import nltk
import openai

# download nltk data
nltk.download("vader_lexicon")
nltk.download("punkt")

# globals
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TEMPERATURE = 0.37
STORY_SAMPLE_SIZE = 10

You'll be able to configure these global variables using an embedded widget form below.


In [ ]:
# this cell focuses on some implemetation details specific to
# this notebook that aren't actually important to understand
# you can just ignore/collapse it if you would prefer
import ipywidgets as pywidgets
import requests as request
import requests_cache
import backoff

# configuration widgets
from widgets.config import (
    modelDropdown,
    apiKeyInput,
    apiKeyUpdateButton,
    temperatureSlider,
    sampleSizeSlider,
    sampleSizeWarningLabel,
    openAiHeader,
    hackerNewsHeader,
)

# project-specific widgets
from widgets.simple import simpleAnalysisWidget
from widgets.advanced import advancedAnalysisWidget, configureOpenAi
from widgets.tokens import tokenAnalysisWidget, configureModel

# project-specific utilities
from utils.obfuscate import obfuscateKey
from utils.array import checkArrayLengths

# we don't want to display too many entries in our DataFrames
# if the sample size is too large
DATAFRAME_LIMIT = 20

# we'll use this session to cache our hacker news api requests
REQUEST_CACHE_EXPIRATION_SECONDS = 60 * 15
session = requests_cache.CachedSession(
    "hackernews_cache", expire_after=REQUEST_CACHE_EXPIRATION_SECONDS
)

## Configuration

You can make changes to the configuration form below at any time and rerun cells that make requests to the OpenAI API or Hacker News API to see how the results change.

You can configure the following values:

- **Open AI API Key**: Your [OpenAI API key](https://platform.openai.com/account/api-keys) is read from the `$OPENAI_API_KEY` environment variable if it's set, but you can override it in this notebook; when you click the **Update Key** button the key you entered will be obfuscated and stored in the `OPENAI_API_KEY` global variable
- **Model**: The [OpenAI model](https://platform.openai.com/docs/models) that the demo should use; you can choose between the `gtp-3.5-turbo` and `gpt-4` models for this demo
- **Temperature**: A model's [temperature](https://platform.openai.com/docs/guides/gpt/how-should-i-set-the-temperature-parameter) is a measure of how "creative" it's response will be; you can set this to `0` for something pretty close to deterministic responses to simple queries
- **Sample Size**: We'll be gathering the top stories from the [Hacker News API](https://github.com/HackerNews/API) and then analyzing the sentiment of a sample of those stories' titles; this controls how large that sample is


In [ ]:
# this code cell is just used to display a widget for us to
# configure some settings that other cells in this notebook rely on
# you can just ignore/collapse it if you would prefer
apiKeyInput.value = obfuscateKey(OPENAI_API_KEY)
sampleSizeSlider.value = STORY_SAMPLE_SIZE
temperatureSlider.value = TEMPERATURE


def updateApiKey(event):
    global OPENAI_API_KEY
    OPENAI_API_KEY = apiKeyInput.value
    apiKeyInput.value = obfuscateKey(OPENAI_API_KEY)


def updateSampleSize(change):
    global STORY_SAMPLE_SIZE
    STORY_SAMPLE_SIZE = change["new"]


def updateTemperature(change):
    global TEMPERATURE
    TEMPERATURE = change["new"]


temperatureSlider.observe(updateTemperature, names="value")
sampleSizeSlider.observe(updateSampleSize, names="value")
apiKeyUpdateButton.on_click(updateApiKey)

apiKeyConfigWidget = pywidgets.HBox([apiKeyInput, apiKeyUpdateButton])
openAiConfigWidget = pywidgets.VBox(
    [openAiHeader, apiKeyConfigWidget, modelDropdown, temperatureSlider]
)
hackerNewsConfigWidget = pywidgets.VBox(
    [hackerNewsHeader, sampleSizeSlider, sampleSizeWarningLabel]
)
configWidget = pywidgets.VBox([openAiConfigWidget, hackerNewsConfigWidget])

display(configWidget)

## Simple sentiment analysis with NLTK

Let's take a look at a simple example of sentiment analysis with `nltk` using the **V**alence **A**ware **D**ictionary and s**E**ntiment **R**easoner ([VADER](https://vadersentiment.readthedocs.io/en/latest/pages/introduction.html)) module.

VADER's `SentimentIntensityAnalyzer` returns an object with positive, negative, and neutral scores for the given text as well as a combined `compound` score computed from the other three.

For this basic example, we're going to rely on the `compound` score and create a naive rating scale that converts that score into a string ranging from `very positive` to `very negative`


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()


def convertSentimentToLabel(sentiment):
    sentimentScore = sentiment["compound"]

    if sentimentScore >= 0.75:
        return "very positive"
    elif sentimentScore >= 0.4:
        return "positive"
    elif sentimentScore >= 0.1:
        return "leaning positive"
    elif sentimentScore <= -0.1 and sentimentScore > -0.4:
        return "leaning negative"
    elif sentimentScore <= -0.4 and sentimentScore > -0.75:
        return "negative"
    elif sentimentScore <= -0.75:
        return "very negative"
    else:
        return "neutral"


def analyzeSentiment(text):
    if not text:
        return ""

    return analyzer.polarity_scores(text)


# some simple test statements for our analyzer
statements = [
    "I love that movie.",
    "I hate that movie.",
    "I like that movie.",
    "I dislike that movie.",
    "I saw that movie.",
]

for statement in statements:
    print(f"{statement} ({convertSentimentToLabel(analyzeSentiment(statement))})")

We've wired the input below up to the same analyzer function from above. Type in some text and see how the analyzer responds.


In [ ]:
# this code cell is just used to display a widget
# that uses the analyzeSentiment function we created
# you can just ignore/collapse it if you would prefer
display(simpleAnalysisWidget)

## How it works

Sentiment analysis, like most text analysis involves a multistep process:

1. **Stemming / Lemmatization**: reduces the words in the text to their root forms to simplify comparison between different forms of the same words
   1. **Stemming**: removes suffixes as an attempt to reduce words to their root forms
   2. **Lemmatization**: uses a morphological analysis of words to reduce them to their root forms
2. **Tokenization**: breaks the text into individual units of meaning called tokens
3. **Vectorization**: converts the tokens into a id that can be used for comparison
4. **Comparison**: compares the tokens to a known set of tokens to determine the sentiment

**Note**: This is a simplification of the process to distill it into an easy to digest format, but it is not a full picture and doesn't include the data gathering, cleaning, and labeling or actual training process.

### Learn more

- [Tokenization, Stemming, and Lemmatization in Python](https://thepythoncode.com/article/tokenization-stemming-and-lemmatization-in-python)
- [Python for NLP: Tokenization, Stemming, and Lemmatization with SpaCy Library](https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/)
- [What is Tokenization in Natural Language Processing (NLP)?](https://www.machinelearningplus.com/nlp/what-is-tokenization-in-natural-language-processing/)
- [Understanding NLP Word Embeddings — Text Vectorization](https://towardsdatascience.com/understanding-nlp-word-embeddings-text-vectorization-1a23744f7223)

### Language models

In this case we're taking advantage of an existing [language model](https://en.wikipedia.org/wiki/Language_model), VADER, that has been trained to analyze sentiment in text, but if we wanted to train our own model, it would be a much more involved process.

With the advent of [Large Language Models](https://en.wikipedia.org/wiki/Large_language_model) (LLMs), like the [Generative Pre-Trained Transformer](https://en.wikipedia.org/wiki/Generative_pre-trained_transformer) (GPT) models that power ChatGPT [large language models have exploded in popularity](https://informationisbeautiful.net/visualizations/the-rise-of-generative-ai-large-language-models-llms-like-chatgpt/).


### LLM family tree

<div style="display: flex; alight-items: center; justify-content: center;"><a href="https://github.com/Mooler0410/LLMsPracticalGuide" target="_blank"><img alt="LLM Evolutionary Tree" src="./assets/llm-family-tree.gif" /><a/></div>

This visualiztion from [Harnessing the Power of LLMs in Practice: A Survey on ChatGPT and Beyond](https://arxiv.org/abs/2304.13712) provides a great overview of how language models have evolved over time and gives you a sense of just how much things have been developing in the last 12 months.


### The power of LLMs

We can leverage the inference and predictive capabilities of these models to perform tasks like sentiment analysis with greater accuracy without having to train our own models.

We can even leverage some prompting techniques - which we'll explore in later cells - to quickly teach the model how to perform more unique analyses and refine our results.

In the past, these would have been a significant undertaking, but now we can acheive similar results with some simple prompting.


## Real world data

Let's take a look at how this works with text generated by other humans (_probably_) without expecting someone would be trying to analyze the sentiment of their text.

For this example, we'll pull in a random sample of the [top stories](https://github.com/HackerNews/API#new-top-and-best-stories) on [Hacker News](https://news.ycombinator.com/) and analyze the sentiment of each submission's title.

You can run the cell below a few times to generate different samples of the top stories until you find a collection you prefer and then rerun the cells after it to use that sample for the rest of the notebook.

**Note**: You can use the configuration widget above to adjust your sample size to find the collection of data that feels right to you.


In [ ]:
import numpy as np


def sampleStories(sampleSize=STORY_SAMPLE_SIZE):
    topStoryIdsRequest = session.get(
        "https://hacker-news.firebaseio.com/v0/topstories.json"
    )

    if topStoryIdsRequest.status_code != 200:
        print("There was a problem getting the top stories from Hacker News")
        exit()

    topStoryIds = topStoryIdsRequest.json()

    storyIds = np.array(topStoryIds)[
        np.random.choice(len(topStoryIds), sampleSize, replace=False)
    ]

    return storyIds


def getStoryDetails(storyId):
    # we'll use the same request cache so that we don't have to request a story's details more than once
    storyRequest = session.get(
        f"https://hacker-news.firebaseio.com/v0/item/{storyId}.json"
    )

    if storyRequest.status_code != 200:
        print(f"There was a problem getting story {storyId} from Hacker News")
        return None
    else:
        story = storyRequest.json()

    return story


def getStories(storyIds):
    stories = {}

    for storyId in storyIds:
        story = getStoryDetails(storyId)

        if "title" in story:
            stories[storyId] = {
                "title": story["title"],
                "time": story["time"],
                "sentiment": {"vader": "", "nrclex": {}, "openai": {}},
            }

    return stories


stories = getStories(sampleStories())

for storyId, story in stories.items():
    print(story["title"])

Let's see what VADER thinks about the sentiment of these titles.


In [ ]:
def analyzeStories(stories):
    for _, story in stories.items():
        story["sentiment"]["vader"] = convertSentimentToLabel(
            analyzeSentiment(story["title"])
        )
        print(f"{story['title']} ({story['sentiment']['vader']})")


analyzeStories(stories)

While this is easy enough to implement and might give us a general idea of the sentiment, what if we want to push things a little further?

What if we have more complex text to analyze or have content that VADER's training doesn't handle well?

We could train our own model, but that's a lot of work.


## ChatGPT

ChatGPT is an LLM that makes use of GPT architecture combined with [Instruction Tuning](https://openai.com/research/instruction-following) to follow instructions and generate text based on the prompts that we provide.

It's training data includes a whole bunch of stuff that we've all posted on the Internet over the years, as well as lots of other content.

This vast trove of training data, combined with the flexibility provided by it's architecture and tuning, gives ChatGPT an impressive ability to respond to our requests for many tasks without needing to be retrained or [fine-tuned](https://www.lakera.ai/insights/llm-fine-tuning-guide) for a specific task.

### How ChatGPT works

In responding to our prompts, ChatGPT follows a similar process to the NLP workflow described above.

It breaks our prompts into [tokens](https://learn.microsoft.com/en-us/semantic-kernel/prompt-engineering/tokens), predicts which tokens should logically follow the ones that we've provided, and returns that text.

ChatGPT's tuning based on Reinforcement Learning from Human Feedback ([RLHF](https://www.assemblyai.com/blog/how-rlhf-preference-model-tuning-works-and-how-things-may-go-wrong/)) is what lead it to be so popular, and is also part of what makes it so powerful.

#### Learn more

- [How ChatGPT Actually Works](https://www.assemblyai.com/blog/how-chatgpt-actually-works/)
- [How ChatGPT Works: The Models Behind The Bot](https://towardsdatascience.com/how-chatgpt-works-the-models-behind-the-bot-1ce5fca96286)
- [The inside story of how ChatGPT was built from the people who made it](https://www.technologyreview.com/2023/03/03/1069311/inside-story-oral-history-how-chatgpt-built-openai/)

### Tokens

Tokenization breaks text down into units of meaning, and just like the stemming/lemmatization that we discussed earlier, you'll notice that words are often broken down into their roots and suffixes when tokenized by ChatGPT's Byte Pair Encoding ([BPE](https://en.wikipedia.org/wiki/Byte_pair_encoding)) tokenization algorithm, [tiktoken](https://github.com/openai/tiktoken).


In [ ]:
import tiktoken


def tokenize(text):
    tokens = []
    ids = []

    # To get the tokeniser corresponding to a specific model in the OpenAI API:
    encoding = tiktoken.encoding_for_model(modelDropdown.value)

    tokenized = encoding.encode(text)

    for tokenId in tokenized:
        ids.append(tokenId)
        tokens.append(encoding.decode_single_token_bytes(tokenId).decode("utf-8"))

    return (tokens, ids)


statements = [
    "I love that movie.",
    "I hate that movie.",
    "I like that movie.",
    "I dislike that movie.",
    "I saw that movie.",
]

for statement in statements:
    (statementTokens, statementIds) = tokenize(statement)
    print(f"{statementTokens} ({len(statementTokens)} tokens)")
    print(f"{statementIds}")
    print("---")

We've wired the input below up to the same tokenizer function above. Type in some text and see how the tokenizer responds.

There's also a great visualizer available at [https://gpt-tokenizer.dev/](https://gpt-tokenizer.dev/).


In [ ]:
# this code cell is just used to display a widget
# that uses the tokenize function we created
# you can just ignore/collapse it if you would prefer
configureModel(modelDropdown.value)

display(tokenAnalysisWidget)

## Prompt engineering

[Prompt engineering](https://en.wikipedia.org/wiki/Prompt_engineering) (or "prompting" if you are into the whole brevity thing) is the process of creating and testing instructions for the model (called "prompts") to find the most concise set of instructions that will guide the model towards returning your desired results as often as possible while minimizing undesired output like [hallucinations](<https://en.wikipedia.org/wiki/Hallucination_(artificial_intelligence)>) and [apologies](https://news.ycombinator.com/item?id=36949931).

In general, each message you send and each response that you receive become part of the overall prompt for the next message, but there are strategies for managing a conversation's memory in order to selectively exclude messages that might lead to the model getting off track if repeated often enough.

You can think of the overall conversation as a document of text - it can help to imagine it as something like a [screenplay](https://en.wikipedia.org/wiki/Screenplay).

There are various types of messages that make up this screenplay:

- **System**: system messages are sort of like stage directions, they describe the overall parameters that the model should follow and provide any other context that the model might need to know about as the conversation continues
- **User**: user messages are the individual prompts that the user sends to the model
- **Assistant**: assistant messages are the responses the model generates to the user's prompts

If you're just chatting with ChatGPT via it's web-based User Interface (UI), you're probably familiar with **User** and **Assistant** messages, but you may not know that there's a **System** message behind the scenes that helps guide how the model responds to your messages.

[Custom Instructions](https://openai.com/blog/custom-instructions-for-chatgpt) are sort of like [system prompts](https://github.com/jujumilk3/leaked-system-prompts), but don't give us quite as much control as we can exercise via the [Chat API](https://platform.openai.com/docs/api-reference/chat) - or as much control as we can get with some of the recent open source models.

### Example conversation document

When you put it all together, the whole thing looks a bit like this:

```
[System]
Assistant is a large language model trained by OpenAI.
Knowledge cutoff: 2021-09
Current date: 2023-08-17
Browsing: disabled

[User]
Who won the Super Bowl in 2022?

[Assistant]
I'm sorry, but I don't have access to real-time information as my knowledge
was last updated in September 2021. To find out the winner of the Super Bowl
in 2022, I recommend checking a reliable sports news website, a sports app,
or conducting a quick internet search for the most up-to-date information.
```

**Note**: There are also messages with the type `function` that indicate that the Assistant would like to take advantage of [function calling](https://openai.com/blog/function-calling-and-other-api-updates) by asking the system to execute the function with the given name and pass it the given parameters, but for this demo, we'll be ignoring those.


### Basic example

Here's an example of a basic prompt we could use for seniment analysis:


In [ ]:
BASIC_SYSTEM_PROMPT = """
You are VibeCheck, an advanced AI system for detecting the sentiment conveyed in user-generated text.

The user will provide you with a prompt, and you will respond with the sentiment of that prompt.

Do not include any punctuation and only use lower case letters.
"""


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def basicChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [{"role": "system", "content": BASIC_SYSTEM_PROMPT}]

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE,
    )

    return response.choices[0].message["content"]

Let's apply this to our Hacker News stories from earlier.


In [ ]:
if OPENAI_API_KEY:
    for storyId, story in stories.items():
        sentiment = basicChatGptSentiment(story["title"])

        if modelDropdown.value not in story["sentiment"]["openai"]:
            story["sentiment"]["openai"][modelDropdown.value] = {}

        story["sentiment"]["openai"][modelDropdown.value]["basic"] = sentiment

        print(f"{story['title']} ({sentiment})")
else:
    print("Please enter your OpenAI API key above and rerun this cell")

### Going further

What if we wanted to dig a bit deeper and consider the emotions that might be associated with some text rather than just a simple positive to negative spectrum?

In the traditional NLP approach, there were tools like [NRCLex](https://pypi.org/project/nrclex/) that could help us with this, too.

Let's explore how we could analyze the emotional content of some text with `nrclex`.


In [ ]:
from nrclex import NRCLex


def getNRCEmotion(text):
    emotion = NRCLex(text)

    return emotion.top_emotions


for storyId, story in stories.items():
    emotions = []

    emotionAnalysis = getNRCEmotion(story["title"])

    for emotion, value in emotionAnalysis:
        if value > 0.00:
            emotions.append(emotion)

    story["sentiment"]["nrclex"] = ", ".join(emotions)

    print(
        f"{story['title']} {('(' + ', '.join(emotions) + ')') if len(emotions) else ''}"
    )

But, with how short some of our titles can be, it doesn't always seem to get good results and it seems like sometimes it disagrees with the VADER sentiment analysis.

Luckily, we can pretty easily adapt our initial prompt to get ChatGPT to do this for us, too.


In [ ]:
ADVANCED_SYSTEM_PROMPT = """
You are VibeCheck, an advanced AI system for detecting the sentiment conveyed in user-generated text.

The user will provide you with a prompt, and you will analyze it following these steps:

1. Analyze the prompt for relevant emotion, tone, affinity, sarcasm, irony, etc.
2. Analyze the likely emotional state of the author based on those findings
3. Summarize the emotional state and sentiment of the prompt based on your findings with at least 2, but no more than 5 names for emotions

Only return the output from the final step to the user.

Only respond with lowercase letters and separate each emotion with a comma and a space
"""


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def advancedChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [{"role": "system", "content": ADVANCED_SYSTEM_PROMPT}]

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE,
    )

    return response.choices[0].message["content"]

Let's apply this to our Hacker News stories from earlier.


In [ ]:
if OPENAI_API_KEY:
    for storyId, story in stories.items():
        sentiment = advancedChatGptSentiment(story["title"])

        if modelDropdown.value not in story["sentiment"]["openai"]:
            story["sentiment"]["openai"][modelDropdown.value] = {}

        story["sentiment"]["openai"][modelDropdown.value]["advanced"] = sentiment

        print(f"{story['title']} ({sentiment})")
else:
    print("Please enter your OpenAI API key above and rerun this cell")

## Comparison of tools

The widget below will allow you to enter arbitrary text and analyze it using the VADER sentiment analysis function from above, the NRCLex emotional analysis function from above, and the ChatGPT emotion analysis prompt we just created.

Play around with it and see how our various tools respond.


In [ ]:
# this code cell is just used to display a widget
# that uses the analyzeSentiment function we created
# as well as the advancedChatGptSentiment function
# you can just ignore/collapse it if you would prefer
configureOpenAi(OPENAI_API_KEY, modelDropdown.value, TEMPERATURE)

display(advancedAnalysisWidget)

## Beyond sentiment

What if we were looking to do something a little more complicated than just basic sentiment or emotion analysis?

What if we wanted to describe the sentiment of some text via an emoji?

**Note**: GPT-4 seems to handle emojis better than GPT-3.5-Turbo, but will incur higher costs.


In [ ]:
EMOJI_SYSTEM_PROMPT = """
You are VibeCheck, an advanced AI system for detecting the sentiment conveyed in user-generated text.

The user will provide you with a prompt, and you will analyze it following these steps:

1. Analyze the prompt for relevant emotion, tone, affinity, sarcasm, irony, etc.
2. Analyze the likely emotional state of the author based on those findings
3. Summarize the emotional state and sentiment of the prompt based on your findings with at least 2, but no more than 5 names for emotions
4. Convert the emotional states from your findings into a representative emoji or group of emojis

Only return the output from the final step to the user.

Repsond with at least 1, but not more than 5, emoji.
"""


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def emojiChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [{"role": "system", "content": EMOJI_SYSTEM_PROMPT}]

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE,
    )

    return response.choices[0].message["content"]

Let's apply this to our Hacker News stories from earlier.


In [ ]:
if OPENAI_API_KEY:
    for storyId, story in stories.items():
        sentiment = emojiChatGptSentiment(story["title"])

        if modelDropdown.value not in story["sentiment"]["openai"]:
            story["sentiment"]["openai"][modelDropdown.value] = {}

        story["sentiment"]["openai"][modelDropdown.value]["emoji"] = sentiment

        print(f"{story['title']}({sentiment})")
else:
    print("Please enter your OpenAI API key above and rerun this cell")

## Prompting strategies

In the previous examples we've been using [Zero Shot](https://www.promptingguide.ai/techniques/zeroshot) prompting, which means we're asking the model to repsond without giving it an example of what kind of response we'd like for it to have.

There are other prompting strategies we can employ, though:

- **One Shot**: gives the model a single example of how we'd like it to respond to guide it's output; this is useful for situations where the model needs a little guidance, but we don't wnat to interfere with how it performs on other tasks
- [**Few Shot**](https://www.promptingguide.ai/techniques/fewshot): gives the model a few examples of how we'd like it to respond to different prompts to help guide it's output; this is useful for situations where the model is doing something novel and needs more guidance, and we're going to be mostly focusing on asking the model to perform the task that we're providing examples for

**Note**: For other types of tasks there are various prompting strategies that can be useful, like [Chain of Thought Reasoning](https://www.promptingguide.ai/techniques/cot), [Directional Stimulus Prompting](https://www.promptingguide.ai/techniques/dsp), and even telling the model to [take a deep breath](https://arstechnica.com/information-technology/2023/09/telling-ai-model-to-take-a-deep-breath-causes-math-scores-to-soar-in-study/) can help it do math.

### Learn more about prompting strategies

- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [Master Prompting Concepts: Zero-Shot and Few-Shot Prompting](https://www.promptengineering.org/master-prompting-concepts-zero-shot-and-few-shot-prompting/)
- [ChatGPT Prompt Engineering Tips: Zero, One and Few Shot Prompting](https://www.allabtai.com/prompt-engineering-tips-zero-one-and-few-shot-prompting/)
- [Tips to enhance your prompt-engineering abilities](https://cloud.google.com/blog/products/application-development/five-best-practices-for-prompt-engineering)


### One shot prompting

Providing a single example of the desired output can help with things like proper formatting and refine the quality of the model's output.


In [ ]:
# Grabbed from https://news.ycombinator.com/ at 2023-09-20 13:00 EDT
# Reference: https://news.ycombinator.com/item?id=37598299
ONE_SHOT_USER_EXAMPLE = (
    "Cisco pulled out of the SentinelOne acquisition after due dilligence"
)

ONE_SHOT_BOT_EXAMPLE = "🤨"


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def oneShotChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [
        {"role": "system", "content": EMOJI_SYSTEM_PROMPT},
        {"role": "user", "content": ONE_SHOT_USER_EXAMPLE},
        {"role": "assistant", "content": ONE_SHOT_BOT_EXAMPLE},
    ]

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE,
    )

    return response.choices[0].message["content"]

Let's apply this to our Hacker News stories from earlier and see how it changes the results.


In [ ]:
if OPENAI_API_KEY:
    for storyId, story in stories.items():
        sentiment = oneShotChatGptSentiment(story["title"])

        if modelDropdown.value not in story["sentiment"]["openai"]:
            story["sentiment"]["openai"][modelDropdown.value] = {}

        story["sentiment"]["openai"][modelDropdown.value]["oneshot"] = sentiment

        print(f"{story['title']}({sentiment})")
else:
    print("Please enter your OpenAI API key above and rerun this cell")

### Few shot prompting

Providing a few examples of desired responses can give the model a chance to learn how you'd like it to respond.

**Note**: Few shot prompting can also lead to issues where the model doesn't respond as creatively or won't perform as well on other tasks, which can be great for certain use cases, but might require a higher temperature setting for others.


In [ ]:
# Grabbed from https://news.ycombinator.com/ at 2023-09-20 13:10 EDT
FEW_SHOT_USER_EXAMPLES = [
    ONE_SHOT_USER_EXAMPLE,
    # Reference: https://news.ycombinator.com/item?id=37595898
    "Atlassian cripples Jira automation for all but enterprise customers",
    # Reference: https://news.ycombinator.com/item?id=37586264
    "Toyota Research claims breakthrough in teaching robots new behaviors",
]

FEW_SHOT_BOT_EXAMPLES = [
    ONE_SHOT_BOT_EXAMPLE,
    "😖",
    "👏",
]


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def fewShotChatGptSentiment(prompt, model=modelDropdown.value):
    messages = [{"role": "system", "content": EMOJI_SYSTEM_PROMPT}]

    for i, userExample in enumerate(FEW_SHOT_USER_EXAMPLES):
        messages.append({"role": "user", "content": userExample})
        messages.append({"role": "assistant", "content": FEW_SHOT_BOT_EXAMPLES[i]})

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=TEMPERATURE,
    )

    return response.choices[0].message["content"]

Let's apply this to our Hacker News stories from earlier and see how it changes the results.


In [ ]:
if OPENAI_API_KEY:
    for storyId, story in stories.items():
        sentiment = fewShotChatGptSentiment(story["title"])

        if modelDropdown.value not in story["sentiment"]["openai"]:
            story["sentiment"]["openai"][modelDropdown.value] = {}

        story["sentiment"]["openai"][modelDropdown.value]["fewshot"] = sentiment

        print(f"{story['title']} ({sentiment})")
else:
    print("Please enter your OpenAI API key above and rerun this cell")

## Comparing approaches

We've looked at various approaches to analyzing sentiment and explored some interesting and novel ways that we can work with AI models like ChatGPT to perform tasks that used to require large investments of time to gather and label data and then train a model.

Let's compare the results of each analysis.


### Gathering our data

We'll start by mapping our data into a format that is easier to display with [DataFrames](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) provided by the [`pandas`](https://pandas.pydata.org/) library.


In [ ]:
# this cell is used to gather our data into an object that's easier to work with
# when displaying some dataframes with slices of what we've explored
import pandas as pd

sentimentData = {
    "Story": [],
    "VADER": [],
    "NRC": [],
    "ChatGPT (Sentiment)": [],
    "ChatGPT (Emotion)": [],
    "Zero Shot": [],
    "One Shot": [],
    "Few Shot": [],
}

for storyId, story in stories.items():
    if "title" in story:
        sentimentData["Story"].append(story["title"])

    if "vader" in story["sentiment"]:
        sentimentData["VADER"].append(story["sentiment"]["vader"])

    if "nrclex" in story["sentiment"]:
        sentimentData["NRC"].append(story["sentiment"]["nrclex"])

    if (
        "openai" in story["sentiment"]
        and modelDropdown.value in story["sentiment"]["openai"]
    ):
        if "basic" in story["sentiment"]["openai"][modelDropdown.value]:
            sentimentData["ChatGPT (Sentiment)"].append(
                story["sentiment"]["openai"][modelDropdown.value]["basic"]
            )

        if "advanced" in story["sentiment"]["openai"][modelDropdown.value]:
            sentimentData["ChatGPT (Emotion)"].append(
                story["sentiment"]["openai"][modelDropdown.value]["advanced"]
            )

        if "emoji" in story["sentiment"]["openai"][modelDropdown.value]:
            sentimentData["Zero Shot"].append(
                story["sentiment"]["openai"][modelDropdown.value]["emoji"]
            )

        if "oneshot" in story["sentiment"]["openai"][modelDropdown.value]:
            sentimentData["One Shot"].append(
                story["sentiment"]["openai"][modelDropdown.value]["oneshot"]
            )

        if "fewshot" in story["sentiment"]["openai"][modelDropdown.value]:
            sentimentData["Few Shot"].append(
                story["sentiment"]["openai"][modelDropdown.value]["fewshot"]
            )

### Sentiment analysis

First let's compare the VADER sentiment analysis to our basic ChatGPT sentiment analysis prompt.


In [ ]:
# this cell is only used to display a dataframe of our sentiment analysis results
try:
    if checkArrayLengths(
        sentimentData["Story"],
        sentimentData["VADER"],
        sentimentData["ChatGPT (Sentiment)"],
    ):
        sentimentDataFrame = pd.DataFrame(
            data=sentimentData,
            columns=["Story", "VADER", "ChatGPT (Sentiment)"],
        )

        display(
            sentimentDataFrame
            if STORY_SAMPLE_SIZE <= DATAFRAME_LIMIT
            else sentimentDataFrame.head(DATAFRAME_LIMIT)
        )
    else:
        print(
            "Error: Different number of stories and sentiment results. Please rerun the VADER, Basic ChatGPT Example, and Gathering Our Data cells above and then rerun this cell."
        )
except NameError:
    print(
        "Error: No sentiment data to display. Please rerun the Gathering Our Data cell above and then rerun this cell."
    )

### Emotion analysis

Next let's compare the emotional analysis of NRCLex to our ChatGPT emotional analysis prompt.


In [ ]:
# this code cell is only used to display a dataframe with our emotional analysis results
try:
    if checkArrayLengths(
        sentimentData["Story"], sentimentData["NRC"], sentimentData["ChatGPT (Emotion)"]
    ):
        emotionDataFrame = pd.DataFrame(
            data=sentimentData, columns=["Story", "NRC", "ChatGPT (Emotion)"]
        )

        # often NRCLex will not have data and instead of displaying NaN we'll leave it blank
        emotionDataFrame = emotionDataFrame.fillna("")

        display(
            emotionDataFrame
            if STORY_SAMPLE_SIZE <= DATAFRAME_LIMIT
            else emotionDataFrame.head(DATAFRAME_LIMIT)
        )
    else:
        print(
            "Error: Different number of stories and sentiment results. Please rerun the NRCLex, Advanced ChatGPT Example, and Gathering Our Data cells above and then rerun this cell."
        )
except NameError:
    print(
        "Error: No emotion data to display. Please rerun the Gathering Our Data cell above and then rerun this cell."
    )

### Prompting strategies

Finally, let's compare the zero shot, one shot, and few shot approaches to our emoji analyzer.


In [ ]:
# this cell is just used to display a dataframe with our emoji results
try:
    if checkArrayLengths(
        sentimentData["Story"],
        sentimentData["Zero Shot"],
        sentimentData["One Shot"],
        sentimentData["Few Shot"],
    ):
        emojiDataFrame = pd.DataFrame(
            data=sentimentData, columns=["Story", "Zero Shot", "One Shot", "Few Shot"]
        )

        display(
            emojiDataFrame
            if STORY_SAMPLE_SIZE <= DATAFRAME_LIMIT
            else emojiDataFrame.head(DATAFRAME_LIMIT)
        )
    else:
        print(
            "Error: Different number of stories and emoji results. Please rerun the Emjoji Classifier, One Shot, Few Shot, and Gathering Our Data cells above and then rerun this cell."
        )
except NameError:
    print(
        "Error: No emoji data to display. Please rerun the Gathering Our Data cell above and then rerun this cell."
    )

## Conclusion

NLP tasks like sentiment analyis used to required significant resources and time, but with the advent of LLMs like ChatGPT and the continued discovery of new prompting strategies to guide these models we can quickly perform complex NLP analyses and teach models to perform novel tasks.
